# Chapter 6 데이터 랭글링

## 1. `readr` 패키지

* `tibble`로 외부 파일을 불러오거나 저장하는 함수를 제공하는 패키지

|**함수**|**설명**|
|:-:|:-:|
|`read_csv()`||
|`read_tsv()`||
|`read_delim()`|임의의 구분자로 구분된 파일 불러오기|
|`read_table()`|공백으로 구분된 파일 불러오기|
|`read_spss()`|(haven 패키지)|
|`read_sas()`|(haven 패키지)|

* `read_csv()` 함수

> * `read.csv()` 함수에 비해 약 10배 정도 빠르며, 좀 더 재현성이 뛰어남
> * 첫 번째 행은 변수 이름(header)으로 사용
> * 매개변수 `col_names` - 변수명(header)이 없는 경우 가용
>   * `col_names = FALSE` - 변수명을 X1부터 Xn까지 순차적으로 넣어줌
>   * `col_names = c("x", "y", "z")`
> * 매개변수 `skip` - 파일의 첫 n행 생략 (유효한 데이터가 n줄 이후부터 있는 경우)
> * 매개변수 `na` - 특정 값을 결측값으로 지정

```R
read_csv(directory, col_names = c("A1", "A2", ..., "An"), skip = 5, na = "9999" )
```

## 2. `dplyr` 패키지

* 데이터 가공, EDA 등에 매우 유용하고 편리한 함수 제공하는 패키지
* 적용 대상에 따른 `dplyr` 함수
    * 행 : `filter(), arrange()`
    * 열 : `select(), mutate(), rename(), relocate()`
    * 집단 : `summarise(), group_by()`

|변수|설명|
|:-:|:-:|
|year|출발년도|
|month|출발 월|
|day|출발 일|
|dep_time|실제 출발 시간(HHMM 또는 HMM)|
|arr_time|실제 도착 시간(HHMM 또는 HMM)|
|sched_dep_time|예정된 출발 시간(HHMM 또는 HMM)|
|sched_arr_time|예정된 도착 시간(HHMM 또는 HMM)|
|dep_delay|출발 지연 시간(분)|
|arr_delay|도착 지연 시간(분)|
|carrier|항공사 약어|
|flight|항공편 번호|
|tailnum|비행기 식별 번호|
|origin|출발지|
|dest|목적지|
|air_time|비행 시간(분)|
|distance|비행 거리(miles)|
|hour|예정된 출발 시간; 시|
|minute|예정된 출발 시간; 분|
|time_hour|예정된 비행 날짜 및 시간(POSIXct date)|

> `str` 대신 `glimpse`를 사용하여 콘솔 크기에 맞춰 출력할 수 있음

### A. 행 관련 함수

`-` `filter()` 함수

```R
library(nycflights13)

#### 120분 이상 연착한 항공편 - arr_delay가 120분 이상인 샘플
flights %>% filter(arr_delay >= 120)

#### 다중 조건 : 1월 1일에 출발한 항공편 - month는 1, day는 1인 샘플
flights %>% filter((month == 1) & (day == 1)) ## 832개
flights %>% filter(month == 1, day == 1) ## 여러 개의 args 입력하면 조건으로 인식함

#### 여러 값 추출 : 3, 5, 8월에 출발한 항공권 -> %in% 사용
flights %>% filter(month %in% c(3, 5, 8))
```

`-` `arrange()` 함수 : 데이터프레임 정렬 수행

> 결측치는 항상 후순위로 정렬
>
> `desc()` : 내림차순 정렬

```R
#### 출발 시간 순으로 정렬
flights %>% arrange(year, month, day, dep_time)

#### 도착 지연 순으로 내림차순 정렬
flights %>% arrange(desc(arr_delay))
```

### B. 열 관련 함수

`-` `select()` 함수 : DB에서 쓰는 `SELECT`와 동일

```R
flights %>% select(x1, x2, x3) ## 특정 변수들 선택
flights %>% select(x1:x5)  ## x1부터 x5까지 변수 이름으로 슬라이싱
flights %>% select(-(x1:x5)) ## x1부터 x5까지 빼고 추출

#### 추출 시 변수 이름 변경까지 됨
flights %>%
  select(dep.time = dep_time,
         arr_time = arr_time)
```

`-` 도우미(helpers) 함수

|함수|설명|
|:-:|:-:|
|`starts_with()`|특정 문자열로 시작하는 이름 매칭|
|`ends_with()`|특정 문자열로 끝나는 이름 매칭|
|`contains()`|특정 문자열을 포함하는 이름 매칭|
|`num_range(perfix, range)`|숫자 접미사를 가지는 일련의 이름 매칭|

```R
### select 도우미 함수
flights %>% select(starts_with("sched")) ## 변수 이름이 "sched"로 시작하는 경우
flights %>% select(ends_with("time"))    ## 변수 이름이 "time"으로 끝나는 경우

flights %>% select(contains("arr"))         ## 변수 이름에 "arr"이 포함되는 경우
## -> carrier라는 의도하지 않은 변수가 들어감
flights %>% select(contains("arr_")) ## 이런 식으로 응용 가능하긴 한데, 좀 그럼
```

`-` `mutate()` 함수

* 데이터셋에 존재하는 변수로부터 **새로운 변수 생성**
* 새로운 변수는 기본적으로 데이터셋 마지막 열에 추가
* `.after`를 통해 해당 변수 뒤에, `.before`를 통해 해당 변수 뒤에 생성된 열을 둘 수 있다.

```R
flights %>% mutate(gain = dep_delay - arr_delay)                      # 출발 지연 - 도착 지연(분)
flights %>% mutate(speed = distance / (air_time/60), .before = 1)     # 첫번째 변수 앞에 추가
flights %>% mutate(hour = air_time / 60, .after = air_time)           # air_time 변수 뒤에 추가
```

* `mutate()`와 자주 쓰이는 함수

|함수|설명|
|:-:|:-|
|`row_number()`|행 번호 반환. 또는 순위를 부여할 때 동순위 발생 시 가장 먼저 등장한 값을 우선순위|
|`min_rank()`|순위 부여(동순위 발생 시 중복된 순위로 매김 : minimum으로 매김)|
|`dense_rank()`|순위 부여(동순위 발생 시 항상 연속된 값을 부여). 위 전부 `NA`는 `NA`로|
|`na_if()`|특정 값을 NA로 변환|
|`coalesce()`|여러 열을 넣었을 때, 첫 번째로 NA가 아닌 값을 반환|
|`if_else()`|단순 조건문(이분법적 분류) -> 연속형 변수 등을 이분법적으로 분류하는 경우|
|`case_when()`|여러 개의 조건을 처리 -> 이분법적 분류가 아닌 다항 분류|
|`recode()`|특정 값을 새로운 값으로 변환|

```R
flights %>%
    mutate(idx = row_number(),
           ranking = min_rank(des(distance)), ## 동순위 중복
           ranking2 = dense_rank(distance)),  ## 동순위 발생해도 연속
           )

flights %>% mutate(dep_delay_na = na_if(dep_delay, 0)) ## 0이면 NA로 변경. ~이면 NA이다
flights %>% mutate(first_non_na = coalesce(dep_delay, arr_delay)) ## 처음으로 NA가 아닌 값을 반환

flights %>% murate(carrier_name = recode(carrier, "AA" = "American Airlines", "DL" = "Delta Airlines"))
```

`-` `rename()` 함수

* 특정 변수 이름 변경
* 명시적으로 언급하지 않은 모든 변수를 유지함(백틱까지 쓰면서)
* 숫자열이 앞에 들어온 변수명 같이 변수 이름 규칙을 위반하는 것을 교체하는 데에 유리

```R
#### select에서와 동일한 사용법
flights %>% rename(dep.time = dep_time,
                   arr.time = arr_time)
```

`-` `relocate()` 함수 : 변수의 위치 변경

```R
flights %>%
  filter(dep_delay <= 10 & dep_delay >= -10) %>%
  arrange(desc(arr_delay)) %>%
  relocate(arr_delay, .after = day) ## day 뒤에 감

flights %>%
  filter(dep_delay <= 10 & dep_delay >= -10) %>%
  arrange(desc(arr_delay)) %>%
  relocate(arr_delay, .before = day) ## day 앞에 감

flights %>% relocate(air_time, distance) ## 기본적으로 앞에 데려옴
flights %>% relocate(carrier:tailnum, .before = day)  ## 콜론 사용 문법은 공유
```

### C. 집단 관련 함수

`-` `summarise()` 함수 == `summarize()`

* 통계량을 계산하여 하나의 행으로 요약
* 여러 개의 다양한 요약 함수를 사용할 수 있음

|함수|설명|
|:-:|:-:|
|`n()`|관측값 개수|
|`n_distinct()`|유니크한 값의 수|
|`first()`|첫 번째 값|
|`last()`|마지막 값|
|`nth()`|n번째 값|
|`sum()`||
|`mean()`||
|`sd()`||
|`IQR()`||
|`quantile()`||

`-` `group_by()` 함수

* 특정 변수를 기준으로 그룹화
* 일반적으로 `summarize()`와 함께 사용
* 여러 변수를 기준으로 그룹화 가능
* `ungroup()` : 그룹화 해제

`-` `count()` 함수 : 도우미 함수

* 변수의 고유값별로 관측값 개수를 반환(빈도표 생성)
* `group_by()` + `summarize(n = n())`을 간결하게 표현한 도우미 함수
* `count(col1, wt = col2)`로 `group_by(col1) + summarize(n = sum(col2))`를 간결하게 표현 가능

### D. 관계형 데이터 : 정형 데이터베이스 관련 내용

* 일반적으로 데이터 테이블이 많이 있고, 관심있는 질문에 대답하기 위해 이들을 결합해야 함
* 여러 데이터 테이블을 총칭하여 관계형 데이터라고 함

`-` 키 Key

* 두 테이블을 연결할 때 사용하는 변수 : 관측값을 고유하게 식별하는 역할을 함
* 기본키 Primary Key : 한 테이블 내에서 각 관측값을 고유하게 식별하는 변수(또는 변수들의 집합) PK
* 외래키 Foreign Key : 다른 테이블의 기본키를 참조하여 두 테이블을 연결하는 변수(또는 변수들의 집합) FK

> 기본키는 반드시 모든 관측값을 유일하게 식별해야 하므로, 실제 중복이 없는지 확인하는 것이 중요
>
> 일부 테이블은 명시적으로 기본키가 정해져 있지 않기도 함
> 
> 일반적으로 기본키와 외래키는 일대다 관계를 형성함

`-` 뮤테이팅 조인 Mutating join

* 관측값을 키로 매칭한 뒤, 한 테이블의 변수를 다른 테이블에 복사하여 결합
* 1개의 내부조인과 3개의 외부조인 방식이 있음

|함수|설명|
|:-:|:-:|
|`inner_join(x, y)`|키가 일치하는 관측값만 결합(양쪽 모두 존재)|
|`left_join(x, y)`|`x`의 모든 관측값을 유지, 일치하는 `y`의 값을 결합|
|`right_join(x, y)`|`y`의 모든 관측값을 유지, 일치하는 `x`의 값을 결합|
|`full_join(x, y)`|모든 관측값을 유지하고, 일치하는 부분은 결합|

```R
x <- tibble(key = c(1, 2, 3),
            value_x = c("x1", "x2", "x3"))
y <- tibble(key = c(1, 2, 4),
            value_y = c("y1", "y2", "y3"))

x %>% inner_join(y) ## 알아서 이름이 같은 변수를 기준으로 매핑해주긴 함
x %>% left_join(y, by = "key")
x %>% right_join(y, by = "key")
x %>% full_join(y, by = "key")
```

* `by`를 지정해주지 않으면 두 테이블에 있는 같은 모든 변수를 사용해서 자연조인
* 조인하려는 테이블의 키에 해당하는 변수 이름이 다를 경우, 조인 문법을 사용

```R
flights2 %>% left_join(airports, by = c("origin" = "faa"))
```

`-` 필터링 조인

* 변수를 결합하지 않고, 관측값 자체를 필터링함 -> 세미 조인. 해시 테이블 같은 거
* 매칭 여부만 중요하며, 어떤 관측값이 매칭되었는지는 중요하지 않음

|함수|설명|
|:-:|:-:|
|`semi_join()`|x의 관측값 중 y와 키가 일치하는 값만 유지|
|`anti_join()`|x의 관측값 중 y와 키가 일치하지 않는 값만 반환|

---
---

여기까지 중간고사 범위

**한글 데이터 불러오기**


```R
mydata <- read_csv("mydata.csv", locale = locale(encoding = "CP949"))
```

## 3. tidyr

### A. wide format과 long format

* wide format
    * 사람이 읽기 쉬운 데이터 구조
    * 통계학에서 다루는 데이터 테이블 구조와 동일한 개념
      > 각 행은 동일한 관측 개체, 각 열은 변수
    * 각 행은 관측 단위를 나타내며, 각 열은 관측 단위에 대한 측정값으로 표현
* long format
    * 컴퓨터가 이해하기 편한 데이터 구조
     > 개체에 대해 회차 별 여러 번 측정 / 개체 별 측정 회차 수가 다름 -> 각 회차 별 분포를 알고 싶을 때는 wide format으로
    * wide format보다 유연하여 데이터 추가 및 삭제가 용이
    * 각 열은 요인(factor)으로 변환된 상태
    * 타이디 데이터임 -> 외워둘 필요는 없다네요

> 특정 열을 기준으로 wide format과 long format을 판단함

### B. 변환

`-` long format으로 변환

* wide format을 long format으로 변환 - `pivot_longer()` 함수
    * 행의 개수를 늘리고, 열의 개수를 줄여서 데이터셋을 길게 만듦
    * long format으로 변환하고자 하는 열을 지정하는 방법은 `dplyr::select()` 함수와 동일
      > 마이너스 기호, `starts_with`, `ends_with`, `contains`를 이용하여 변수 지정 가능
    * 기본적으로 이름에 해당하는 열은 문자열로 변환됨
    * `gather()`, `reshape2::melt()` 함수와 동일

```R
pivot_longer(data, cols = c(col1, col2),
             names_to = "name_col", names_prefix = "prefix", ## 접두사 제거
             values_to = "value_col", values_drop_na = TRUE) ## 결측값 제거

mutate(year = parse_integer(year)) ## 이걸로 문자열을 정수형으로 바꿔줄 수 있음
```

`-` wide format으로 변환 : `pivot_wider()`

* 변수로 늘릴 column : `names_from`
> 특정 열에 대해서 값이 없는 경우 알아서 결측값으로 들어감
* 변수를 없애서 셀 안의 값으로 바꿀 column : `values_from`
> 여러 개의 변수를 지정하면 `names_from`의 유니크한 개수 $\times$ 변수 개수만큼의 변수가 생성됨

```R
pivot_wider(data,
            names_from = name_cols,
            values_from = value_cols, values_fill = "NA 처리값" )
```

### C. 열의 분리 및 결합

`-` 열의 분리 : `separate()`

* 단일 값 원칙을 만족하지 못하는 경우
* 한 개의 열에 여러 값들이 결합되어있는 경우, 분리

```R
separate(complx_col, into = c("col1", "col2"), sep = "분리 기준 문자열" )
```

`-` 열의 결합 : `unite()`

* 기본 구분자는 `"_"`

```R
unite(col = new_col, col1, col2, ..., sep = "" )
```

### D. 결측값 처리 - `is.na()`

* `NA` == `not available`
* 결측값은 전염성을 가짐 -> 결측값이 하나라도 존재할 경우 연산 시 결측값으로 반환됨
> 산술연산자와 비교연산자 모두 NA를 반환함
>
> `NA`와 `NA`를 같다고 할 수 있는가? -> 모름. 그니까 `NA`로 반환

`-` 명시적/암묵적 결측값

* 명시적 결측값 - `NA`로 표시된 값
* 암묵적 결측값 - 데이터셋에 존재하지 않는 값

```R
# 2015년 4분기 수익은 NA로 표시되어 있으므로, 명시적 결측값임
# 2016년 1분기 수익은 데이터셋이 존재하지 않으므로, 암묵적 결측값임
stocks <- tibble(
  year   = c(2015, 2015, 2015, 2015, 2016, 2016, 2016),
  qtr    = c(   1,    2,    3,    4,    2,    3,    4),
  return = c(1.88, 0.59, 0.35,   NA, 0.92, 0.17, 2.66))

stocks %>%
  pivot_wider(names_from = year,                  # year를 열로 pivot하여 명시적으로 만듦
              values_from = return) %>%
  pivot_longer(cols = c(`2015`, `2016`),          # 명시적 결측값이 중요하지 않다면
               names_to = "year",                 # 암묵적으로 만듬(NA 제거)
               values_to = "return",
               values_drop_na = TRUE)
```

`-` 결측값 처리 함수

|**함수**|**설명**|
|:-:|:-|
|`complete(col1, col2, ...)`|해당 범주형 열들의 조합에 대하여 명시적 결측값 생성|
|`fill(col1, col2, ...)`|결측값을 가장 최근에 존재하는 값(이월된 마지막 관측값)으로 대체|
|`drop_na(col1, col2, ...)`|결측값이 포함된 행 제거. 파라미터 추가 입력 시 몇개 열에 대해서만 적용 가능|
|`replace_na(list(col1 = "", col2 = 0, ...))` <br> `replace_na(vector, replace_txt)`|결측값을 사용자가 지정한 값으로 대체|

## **4. stringr 패키지**

---

### **A. 문자열 기초**

> 함수들은 문자열이 아니라도 사용 가능함

`-` `str_length()` : 문자열의 길이 확인

```R
babynames %>%
  count(name_length = str_length(name), wt = n)
```

> `str_length(col)` 자체를 생성된 열처럼 바로 사용할 수 있음

`-` `str_sub()` : 서브 스트링

```R
str_sub(x, 1, 3)
str_sub(x, -3, -1) ## 뺴는 게 아니라 파이썬 인덱스처럼 뒤에서부터
```

`-` `str_to_lower()`, `str_to_upper()` : 대소문자 변환

### **B. 데이터에서 문자열 생성**

`-` `str_c(a, b, sep = "")` : 두 개 이상의 벡터를 결합, 구분자 지정 가능

`-` `str_c(v, collapse = "")` : 벡터를 단일 문자열로 결합

* `letters`, `LETTERS` : a-z / A-Z가 저장되어 있음

* `str_replace_na(NA)` : `NA`를 `"NA"`로 변경

`-` `str_glue()` : 고정 문자열과 변수를 직관적으로 결합할 수 있도록 지원. 그냥 f-string

> `{}` 안에 작성한 표현식은 문자열 외부처럼 평가되어 값으로 치환됨
>
> 함수 내부 문자열 작성 시 외부를 묶은 따옴표와 다른 따옴표를 사용

```R
str_glue("Today is {str_to_upper('monday')}") ## 내부는 다른 따옴표로...
str_glue('Today is {str_to_upper("monday")}')
```

`-` `str_glue_data()` : 데이터 프레임에 특화된 `str_glue()`

* `str_glue()` 함수와 유사하지만, 데이터프레임을 명시적으로 참조하여 문자열을 생성
* `str_glue()` 함수보다 더 빠르고, 메모리 관점에서 효율적이며, 벡터화된 처리로 대규모 데이터 처리에 적합

```R
## 열 이름을 변수처럼 사용
info <- tibble(name = c("Kim", "Lee", "Park"),
               age = c(25, 30, 42),
               city = c("Seoul", "Jeonbuk", "Gwangju"))

info %>% str_glue_data("{name} is {age} years old and lives in {city}" )
```

`-` `str_flatten()` : 문자 벡터의 각 원소를 하나의 문자열로 결합

* `str_c(v, collapse = "")`랑 똑같은 거 아님? -> 아니더라
* `summarize()`와 함께 사용하여 문자열을 요약하는 데에 사용 가능

```R
### 마지막 원소를 다르게 결합
str_flatten(c("x", "y", "z"), collapse = ", ", last = ", and ") ## 마지막 원소는 다르게 결합

### 하나의 셀에 여러 개의 데이터를 삽입
mytbl <- tibble(name = c("Carmen", "Carmen", "Marvin", "Terence", "Terence", "Terence"),
                fruit = c("banana", "apple", "nectarine", "cantaloupe", "papaya", "madarine"))

mytbl %>%
  group_by(name) %>%
  summarise(fruits = str_flatten(fruit, ", "))
```

### **C. 정규 표현식 종류**

`-` 정규 표현식

* 문자열의 규칙을 표현하는 특수한 언어
* 일정한 규칙을 만족하는 문자열을 탐색하거나 추출, 변경하는 데 사용
* `str_view()` : 정규 표현식이 문자 벡터에 어떻게 매칭되는지 시각적으로 보여줌
* 여러 단어를 앵커에 먹이거나 반복시키고 싶을 경우 `()`를 사용
* or 연산자 `|`를 사용할 수 있음

|**표현식**|**설명**|
|:-:|:-|
|`.`|어떤 문자 한 개와 일치(줄 바꿈 제외)|
|`^`|`^` 뒤의 표현식이 문자열의 시작과 일치. 맨 처음이라는 것을 지시|
|`$`|`$` 앞의 표현식이 문자열의 끝과 일치. 맨 끝이라는 것을 지시|
|`?`|`?` 앞의 표현식이 0번 또는 1번 일치|
|`+`|`+` 앞의 표현식이 1번 이상 반복되어 일치|
|`*`|`*` 앞의 표현식이 0번 이상 반복되어 일치|
|`()`|표현식을 하나의 집단에 적용|
|`\`|escape 문자열. 정규 표현식을 문자열이 아닌 기호 자체로 변환|

`-` 앵커 : `^, $`

> 문자열 자체가 `"^ ... $"`로 만들어졌다고 생각하면 편함

`-` 반복 : `?, +, *` / `{n,m}`

```R
x <- "MDCCCLXXXVIII"

str_view(x, "C{2}") ## 2번만
str_view(x, "C{2,}") ## 2번 이상
str_view(x, "C{2,5}") ## 2~5번
```

`-` 그룹화 : `()`

> or 연산자 `|`를 사용하여 몇 개의 단어 처리 가능

```R
x <- c("gray", "grey")

str_view(x, "gr(a|e)y")
```

`-` escape 문자열 : `\`

* `writeLines()`를 사용하여 정규표현식이 포함된 텍스트가 실제 어떻게 산출되는지 파악 가능

```R
writeLines("a\\b") ## \로 \를 무력화

str_view("\"", "\"") ## "와 '는 \로 무력화됨


str_view("a\\b", "\\\\") ## \\를 찾기 위해서 \로 \를 무력화 두번
str_view(c("abc", "a.c", "bef"), "a\\.c") ## 정규 표현식 자체를 찾으려면 \\. -> "\."
```

`-` 문자 집합과 매칭

* `[abcd]` : `[]`안에 있는 문자와 일치
* `[^abcd]` : `[]`안에 있는 문자를 제외하고 일치

`-` 자주 사용되는 문자 클래스

* `\d` : 임의의 숫자와 매칭. decimal
* `\s` : 임의의 여백 문자(공백, 탭, 줄바꿈)와 매칭. whiteSpace

### **D. 정규 표현식을 활용한 문자열 패턴 매칭**

`-` `str_detect(x, "reg")` : 문자열에 특정 패턴이 존재하는지 확인. 존재하면 `TRUE`

> `filter()`와 결합하여 조건에 맞는 문자열을 가지는 행을 추출 가능
>
> `group_by` -> `summarize`를 이용하여 범주 별 문자열 빈도에 따른 통계량 산출 가능

```R
library(babynames)

## "x"로 끝나는 이름이 있는 행 추출
babynames %>%
    filter(str_detect(name, "x$"))

## 연도별로 "x"로 끝나는 고유한 이름의 비율
babynames %>%
    group_by(year) %>%
    summarize(
        rate = mean(str_detect(name, "x$"))
    )
```

`-` `str_subset(x, "reg")` : 패턴이 일치하는 문자열 벡터를 반환

`-` `str_count(x, "reg")` : 문자열 별로 패턴이 일치하는 경우가 몇 번 있는지 벡터로 반환

```R
### 이름에서 자음의 개수와 모음의 개수 출력
babynames %>%
  count(name) %>%
  mutate(vowels = str_count(name, "[AEIOUaeiou]"),
         consonants = str_count(name, "[^AEIOUaeiou]"))
```

`-` `str_extract(x, "reg")` : 문자열에서 일치하는 텍스트를 추출

```R
colors <- c("red", "orange", "yellow", "green", "blue", "purple")
color_match <- str_c(colors, collapse = "|")

str_extract(sentences, color_match)
```

`-` `str_extract_all(x, "reg", simplify = FALSE)` : 문자열에서 일치하는 텍스트를 리스트로 전부 추출

```R
has_color = str_subset(sentences, color_match)

str_extract_all(has_color, color_match) ## 리스트로 반환
str_extract_all(has_color, color_match, simplify = TRUE) ## 행렬로 반환
```

`-` `str_replace(x, "reg", "replace")` : 일치하는 첫번째 문자열을 특정 문자열로 변환

`-` `str_replace_all(x, "reg", "replace")` : 일치하는 문자열 전부를 특정 문자열로 변환

```R
### 벡터를 이용해서 여러 패턴을 한번에 변경 가능
x <- c("1 house", "2 cars", "3 people")
str_replace_all(x, c("1" = "one", "2" = "two", "3" = "three"))
```

`-` `str_split(x, "sep")` : 문자열을 특정 구분자를 매개로 분할

```R
str_split(sentences, " ")
str_split(sentences, " ", simplify = TRUE) ## 관측값 별로 행렬 반환
```

## **5. forcats 패키지**

### **A. 팩터 수준(levels) 재정렬**

> 팩터의 `levels`는 입력 시 기본적으로 사전순으로 정렬됨

`-` `fct_inorder(fct)` : `levels`를 입력한 순서대로 재정렬

```R
fct1 <- factor(c("b", "b", "a", "c", "c", "c"))
fct1 ## 기본적으로 levels는 사전순

fct_inorder(fct1) ## 입력순으로 재정렬
```

`-` `fct_infreq(fct)` : `levels`를 각 수준의 빈도가 큰 순서대로 재정렬

```R
fct1 <- factor(c("b", "b", "a", "c", "c", "c"))
fct_inorder(fct1)                                     # Levels: b a c
fct_infreq(fct1)                                      # Levels: c b a
```

`-` `fct_inseq(fct)` : `levels` 자체를 수치값으로 나타낼 수 있을 때, 그 순서대로 정렬(ASC)

```R
fct2 <- factor(1:3, levels = c("2", "3", "1"))
fct_inseq(fct2)                                       # Levels : 1 2 3
```

`-` `fct_rev(fct)` : `levels`의 순서를 뒤바꿈

```R
fct_rev(fct1)                                         # Levels : c b a
fct_rev(fct2)                                         # Levels : 1 3 2

# fct_infreq(), fct_rev() 함수를 조합하여 marital 수준을 빈도 기준으로 내림차순 정렬
gss_cat %>%
  mutate(marital = marital %>% fct_infreq() %>% fct_rev()) %>%
  ggplot(aes(marital)) + geom_bar()
```

`-` `fct_reorder(fct1, num1, .desc = FALSE)` : 특정 변수를 기준으로 팩터 수준을 재정렬

```R
summary1 <- gss_cat %>%
  group_by(marital) %>%
  summarise(tvhours = mean(tvhours, na.rm = TRUE),
            n = n()) %>%
  mutate(marital = fct_reorder(marital, tvhours))
```

`-` `fct_relevel(tbl, fct1, level_value)` : 지정한 수준을 앞쪽(처음)으로 이동하여 재정렬

```R
summary2 <- gss_cat %>%
  group_by(rincome) %>%
  summarize(age = mean(age, na.rm = TRUE),
            n = n())

ggplot(summary2, aes(x = fct_relevel(rincome, "Not applicable"), y = age)) +
  geom_bar(stat = "identity") +
  coord_flip() +
  theme(axis.text = element_text(size = 15))
```

> 결국 시각화 때문에 바꾸는 경우가 많음.

### **B. 팩터 수준(levels) 변경**

> **서로 다른 범주를 잘못 묶으면 결과가 왜곡될 수 있음에 유의**
>
> **순서형 자료 등의 경우 적은 것들을 통합할 수 없고, 인접한 구간을 통합해야 함**

`-` `fct_recode(tbl, fct, "A" = "a", "B" = "b", "B" = "BB", ...)`
* 명시적으로 언급되지 않은 수준은 그대로 유지됨
* 여러 수준을 하나로 묶으려면 동일한 새 수준에 할당
* 매개변수로 입력된 문자열 레벨을 앞에 있는 문자열로 만듦

```R
### 몇개를 병합... (나머지를 Other로 통합)
gss_cat %>%
  mutate(partyid = fct_recode(partyid,
                              "Republican, strong"    = "Strong republican",
                              "Republican, weak"      = "Not str republican",
                              "Independent, near rep" = "Ind,near rep",
                              "Independent, near dem" = "Ind,near dem",
                              "Democrat, weak"        = "Not str democrat",
                              "Democrat, strong"      = "Strong democrat",
                              "Other"                 = "No answer",
                              "Other"                 = "Don't know",
                              "Other"                 = "Other party")) %>%
  count(partyid)
```

`-` `fct_collapse(tbl, fct, A = c("a", "AA"), B = c("b", "BB"), ...)`

* 여러 수준을 하나의 집단으로 병합
* 새 수준에 대해 기존 수준들의 벡터를 지정
* 명시적으로 언급되지 않은 수준은 그대로 유지됨

```R
gss_cat %>%
  mutate(partyid = fct_collapse(partyid,
                                other = c("No answer", "Don't know", "Other party"),
                                rep = c("Strong republican", "Not str republican"),
                                ind = c("Ind,near rep", "Independent", "Ind,near dem"),
                                dem = c("Not str democrat", "Strong democrat"))) %>%
  count(partyid)
```

`-` `fct_lump()`

* 상대적으로 빈도가 낮은 수준을 묶어 `"Other"`로 병합
* 매개변수 `n` : `"Other"`를 제외한 수준의 개수 설정. 몇 개의 수준을 멀쩡히 남길 것인지
* 매개변수 `prop` : `"Other"`로 묶을 수준의 전체 비중 기준을 설정. 넘으면 남기고, 안넘으면 싸잡고...
* 매개변수 `other_level` : `"Other"`대신 들어갈 단어 지정

```R
### 빈도가 가장 높은 상위 10개의 종교만 남김
gss_cat %>%
  mutate(relig = fct_lump(relig, n = 10)) %>%
  count(relig, sort = TRUE) ## 기존에 Other가 이미 있어서 10개만


### Other 대신 다른 표현으로
gss_cat %>%
  mutate(relig = fct_lump(relig, n = 10, other_level = "Others...")) %>%
  count(relig, sort = TRUE) ## 11개 나옴


### 점유율이 0.05 미만인 종교만 남김
gss_cat %>%
  mutate(relig = fct_lump(relig, prop = 0.05)) %>%
  count(relig, sort = TRUE)
```

## **6. lubridate 패키지**

* 날짜/시간을 다루는 함수를 제공
* `tidyverse` 핵심에 포함되지 않으므로, 별도의 import 과정 필요...? 버전마다 다른가봄. 오류가 있던가.

### **A. 날짜/시간 유형**

|**유형**|**설명**|
|:-:|:-|
|date|- 날짜를 나타내며, tibble에서 `<date>`로 출력|
|time|- 하루 중 시간을 나타내며, tibble에서 `<time>`로 출력|
|date-time|- 날짜와 시간을 나타내며, tibble에서 `<dttm>`로 출력 <br>- 시점을 고유하게 식별|

### **B. 문자열(string)로부터 생성**

* 문자열에서 연(year), 월(month), 일(day)의 순서를 파악한 후, “y”, “m”, “d”를 그 순서대로 함수 이름에 사용
* `date-time`을 생성하려면 함수 이름에 `_`와 “h”, “m”, “s” 중 하나 이상을 추가함

```R
library(lubridate)
ymd("2017-01-31")
mdy("January 31st, 2017")
dmy("31-Jan-2017")

ymd_hms("2017-01-31 20:11:59")
mdy_hm("01/31/2017 08:01")
```

### **C. 개별 구성성분(열)으로부터 생성**

`-` `make_datetime(...)` : 여러 값들로 하나의 시간 자료를 생성

```R
### 간단히...
flights %>%
  select(year, month, day, hour, minute) %>%
  mutate(departure = make_datetime(year, month, day, hour, minute))


### hour와 minute를 각각 분리하여 추출한 후, date-time 생성
make_datetime_100 <- function(year, month, day, time) {
  return(make_datetime(year, month, day, time %/% 100, time %% 100))
}

flights_dt <- flights %>%
  filter(!is.na(dep_time), !is.na(arr_time)) %>% ## 결측치 제거
  mutate(dep_time = make_datetime_100(year, month, day, dep_time),
         arr_time = make_datetime_100(year, month, day, arr_time),
         sched_dep_time  = make_datetime_100(year, month, day, sched_dep_time),
         sched_arr_time = make_datetime_100(year, month, day, sched_arr_time)
         )
```

### **D. 기존의 날짜/시간으로부터 생성**

* `date`와 `datetime`간 상호 전환

```R
as_datetime(today())
as_date(now())
```

### **E. 날짜/시간 유형**

|함수|설명|
|:-:|:-|
|`year()`|연도(year) 추출|
`month()`|월(month) 추출
`week()`	|주(week) 추출
`day()`	|일(day) 추출
`hour()`	|시(hour) 추출
`minute()`	|분(minute) 추출
`second()`	|초(second) 추출
`yday()`	|해당 연도의 몇 번째 일(day)인지 반환
`mday()`	|해당 월의 몇 번째 일(day)인지 반환
`wday()`	|해당 주의 무슨 요일(1=일요일, 7=토요일)인지 반환

`-` `month()`, `wday()` 함수 관련 매개변수

* 매개변수 `label = TRUE`이면 문자열, FALSE이면 숫자로 표시 : 디폴트는 `FALSE`
* 매개변수 `abbr = FALSE`이면 이름 전체 반환(label이 FALSE인 경우 무시) : 디폴트는 `TRUE`

### **F. 시간 범위 time spans**

|유형|설명|
|:-:|:-|
`duration`|	초(second) 단위의 정확한 시간 길이를 나타냄
`period`	|주(week), 월(month) 등 사람의 단위(human units)로 표현함
`interval`	|시작 시점과 종료 시점을 나타냄

> date에 뺄셈을 적용하면 difftime이 생성되며, 이는 초(second), 분(minute), 시(hour), 일(day) 또는 주(week) 단위로 시간 범위를 기록하므로 작업이 어려움

`-` `duration` : 사람 기준에선 직관적이지 않을 수 있음

* 항상 초단위를 사용하여 시간 범위를 나타냄.
* 덧셈, 뺄셈, 곱셈 연산이 가능함
> 윤년, 일광 절약 시간제 등의 규칙에 영향을 받기 때문에 시간/월/년과 같은 큰 시간 단위로 특정한 값에 대해 연산 결과가 실제와 항상 같지 않을 수 있음

```R
### 일광 절약 시간제
one_pm <- ymd_hms("2025-03-08 13:00:00", tz = "America/New_York") ## time zome?
one_pm ## EST
one_pm + ddays(1) ## 하루만 더했는데 1시간 더 증가
```